In [1]:
import pandas as pd
file_path = '/kaggle/input/orig-data/TenderHack_msk.xlsx'
df_cte = pd.read_excel(file_path, sheet_name=0)
# df_cte.to_csv('cte.csv', index=False)
df_characteristics = pd.read_excel(file_path, sheet_name=1)
# df_characteristics.to_csv('characteristics.csv', index=False)
grouped_characteristics = (
    df_characteristics
    .groupby('ID конечной категории Портала')
    .apply(lambda x: dict(zip(x['ID характеристики'], x['Название характеристики'])))
    .reset_index(name='Список характеристик')
)
result_df = pd.merge(df_cte,grouped_characteristics, on='ID конечной категории Портала', how='inner')
new_columns = ['ID', 'CTE_name', 'Image_Link', 'Category_ID', 'Category_Name', 'Model', 'Manufacturer', 'Characteristics_List']
result_df.columns = new_columns
cat_char_df = result_df[['Category_ID','Category_Name','Characteristics_List']]
grouped_cat_char_df = (
    cat_char_df
    .groupby(['Category_ID', 'Category_Name'], as_index=False)
    .agg({'Characteristics_List': lambda x: list(x)})
    .rename(columns={'Characteristics_List': 'Aggregated_Characteristics_List'})
)


/tmp/ipykernel_33/594748799.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: dict(zip(x['ID характеристики'], x['Название характеристики'])))


In [50]:
!pip install clickhouse-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 16.0 MB/s eta 0:00:00a 0:00:01


In [51]:
import clickhouse_driver

In [ ]:
# import pandas as pd
# from collections import Counter
# import ast
# def count_characteristics(row):
#     # Преобразовываем строку списка словарей в список словарей
#     characteristics_list = ast.literal_eval(row['Aggregated_Characteristics_List'])
#     characteristics_counter = Counter()
    
#     # Считаем, сколько раз каждая характеристика встречается в списке словарей
#     for char_dict in characteristics_list:
#         for char_name in char_dict.values():
#             characteristics_counter[char_name] += 1
    
#     # Возвращаем список названий характеристик, отсортированный по убыванию популярности
#     return [char_name for char_name, count in characteristics_counter.most_common(10)]

# # Применяем функцию ко всем строкам и создаем новый столбец 'Top_Characteristics'
# grouped_cat_char_df['Top_Characteristics'] = grouped_cat_char_df.apply(count_characteristics, axis=1)


In [27]:
grouped_cat_char_df

,Category_ID,Category_Name,Aggregated_Characteristics_List
0,1100077,Конверты почтовые,"[{214645261: 'Высота конверта', 206594882: 'Oк..."
1,1100142,Наградная продукция,"[{247994787: 'Габариты таблички', 247994785: '..."
2,1103245,Конфеты,"[{227516191: 'Тип продукта', 226202804: 'Марме..."
3,1103367,Зеркала интерьерные,"[{244347787: 'Форма пoлотна', 239516431: 'Форм..."
4,1116434,Смеси сухие белковые композитные,[{206728326: 'Вид смеси энтеральной по содержа...
...,...,...,...
7362,793410082,ФУТЛЯР-УКЛАДКИ/УКЛАДКИ И МЕДИЦИНСКИЕ НАБОРЫ,"[{230342171: 'Лоток полимерный', 230342163: 'У..."
7363,793410084,КАТЕТЕРЫ ДЛЯ ГЕМОДИАЛИЗА,"[{250003913: 'Одноразовый', 250003915: 'Для жи..."
7364,793410086,ФИЛЬТРЫ ДЛЯ СИСТЕМЫ АФЕРЕЗА,"[{239525313: 'Кратность применения', 239525315..."
7365,793410088,ГЕМОСОРБЕНТЫ СЕЛЕКТИВНЫЕ ДЛЯ УДАЛЕНИЯ ЭНДОТОКС...,"[{239614203: 'Сорбционная емкость', 239614201:..."


In [40]:
from collections import Counter
import re

trans_dict = {
    'a': 'а', 'e': 'е', 'o': 'о', 'p': 'р', 'c': 'с', 'y': 'у', 'x': 'х',
    'A': 'А', 'E': 'Е', 'O': 'О', 'P': 'Р', 'C': 'С', 'Y': 'У', 'X': 'Х'
}

def create_translation_table(dictionary):
    return str.maketrans(dictionary)

translation_table = create_translation_table(trans_dict)

def standardize_and_replace(s, translation_table):
    s = s.translate(translation_table)  
    s = re.sub(r'[^\w\s]', '', s)      
    s = s.lower()                     
    s = re.sub(r'\s+', ' ', s)         
    s = s.strip()                     
    return s
final_top =[]
for i in range(grouped_cat_char_df.shape[0]):
    arr = [standardize_and_replace(value, translation_table) for value in grouped_cat_char_df['Aggregated_Characteristics_List'][i][0].values()]
    counter = Counter(arr)
    top_characteristics = counter.most_common(10)
    top_characteristics_values = [item[0] for item in top_characteristics]
    final_top.append(top_characteristics_values)

In [44]:
assert len(final_top) == len(grouped_cat_char_df)

* 

In [45]:
grouped_cat_char_df['top_10_char'] = final_top

In [47]:
grouped_cat_char_df.drop('Aggregated_Characteristics_List',axis=1,inplace=True)

In [48]:
grouped_cat_char_df

,Category_ID,Category_Name,top_10_char
0,1100077,Конверты почтовые,"[окно, защита от попыток незаметного вскрытия ..."
1,1100142,Наградная продукция,"[размер изделия, количество в упаковке, размер..."
2,1103245,Конфеты,"[соответствует требованиям госта, ширина, глуб..."
3,1103367,Зеркала интерьерные,"[ширина, форма полотна, высота, длина, тип рег..."
4,1116434,Смеси сухие белковые композитные,"[объем, пищевые волокна, осмолярность, энергет..."
...,...,...,...
7362,793410082,ФУТЛЯР-УКЛАДКИ/УКЛАДКИ И МЕДИЦИНСКИЕ НАБОРЫ,[бинт марлевый медицинский стерильный 5 м х 10...
7363,793410084,КАТЕТЕРЫ ДЛЯ ГЕМОДИАЛИЗА,"[диаметр катетера, диаметр, расширитель, в наб..."
7364,793410086,ФИЛЬТРЫ ДЛЯ СИСТЕМЫ АФЕРЕЗА,"[масса, стерильность, принцип действия, площад..."
7365,793410088,ГЕМОСОРБЕНТЫ СЕЛЕКТИВНЫЕ ДЛЯ УДАЛЕНИЯ ЭНДОТОКС...,"[объем заполнения кровью, сорбционная емкость,..."


In [57]:
grouped_cat_char_df.to_csv('top_10_char.csv',index=False)

In [55]:
!pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.2/965.2 kB 17.3 MB/s eta 0:00:00a 0:00:01


In [56]:
import clickhouse_connect
client = clickhouse_connect.get_client(
    host='x2ar8i584r.europe-west4.gcp.clickhouse.cloud',
    port=8443,
    username='default',
    password='Avk7dO5kAfK_s'  
)

for index, row in grouped_cat_char_df.iterrows():
    update_query = f"""
    ALTER TABLE tender_hack_table UPDATE top_10_char = {row['top_10_char']}
    WHERE ID = {row['Category_ID']}
    """
    client.query(update_query)


KeyboardInterrupt: 